In [7]:
#!pip install altair pandas numpy
import altair as alt
import pandas as pd
import numpy as np
# norms
# mean sj: (1,1,0,0,0,0,1,1)
# nice sj: (1,1,0,1,0,1,1,1)

ebnorms_b = ["(1,1,0,0,0,0,1,1)", "(1,1,0,1,0,1,1,1)", "(1,1,1,0,1,0,1,1)", "(0,0,0,0,1,1,1,1)", "(0,1,0,1,1,1,1,1)", "(0,1,1,1,0,1,1,1)",
          "(1,1,0,0,1,1,1,1)", "(1,1,0,1,1,1,1,1)", "(0,0,0,0,0,0,1,1)", "(0,1,0,1,0,1,1,1)", "(0,0,0,1,0,1,1,1)", "(0,0,1,1,0,1,1,1)"]

ebnorms_n = ["SJ normal", "SJ nice", "SJ Mean", "IS normal", "IS nice", "IS weird",
             "SS normal", "SS nice", "SH normal", "SH normal", "Emotion-Strict IS", "Emotion-Lenient IS"]

norms_simple_b = ["(1,0,0,1)", "(1,0,1,1)", "(0,0,0,1)", "(0,0,1,1)"]

norms_simple_n = ["SJ", "SS", "SH", "IS"]

columns = ["EBNorm", "Norm", "z", "g", "mu", "chi", "eps", "alpha", "gamma", "pdx", "acr", "B", "G", "AllD", "Disc", "pDisc", "AllC", "m", "n"]

def load_and_process() -> pd.DataFrame: 
    data: pd.DataFrame = pd.read_csv("outputs/results.txt", sep="\t", header=None, names=columns, index_col=False)
    data.eps = data.eps / data.z
    data.chi = data.chi / data.z
    data.alpha = data.alpha / data.z
    data.dropna()
    data.EBNorm = data.EBNorm.replace(ebnorms_b, ebnorms_n)
    data.Norm = data.Norm.replace(norms_simple_b, norms_simple_n)
    #data = data.tail(50000)
    return data

def filter(frame: pd.DataFrame, feature: str, condition) -> pd.DataFrame:
    return frame[frame[feature]==condition]
    
data = load_and_process()


In [8]:
data

,EBNorm,Norm,z,g,mu,chi,eps,alpha,gamma,pdx,acr,B,G,AllD,Disc,pDisc,AllC,m,n
0,Emotion-Strict IS,IS,50.0,2500.0,0.1,0.00002,0.00002,0.02000,0.8,True,75.380910,0.00,1.00,0.00,1.0,0.0,0.00,0.00,1.00
1,Emotion-Strict IS,IS,50.0,2500.0,0.1,0.00002,0.00002,0.00002,0.0,True,3.574434,1.00,0.00,1.00,0.0,0.0,0.00,0.00,1.00
2,Emotion-Strict IS,IS,50.0,2500.0,0.1,0.00002,0.00002,0.00002,0.0,True,0.230539,1.00,0.00,1.00,0.0,0.0,0.00,0.00,1.00
3,Emotion-Strict IS,IS,50.0,2500.0,0.1,0.00002,0.00002,0.00002,0.1,True,38.567796,1.00,0.00,1.00,0.0,0.0,0.00,0.00,1.00
4,Emotion-Strict IS,IS,50.0,2500.0,0.1,0.00002,0.00002,0.00002,0.1,True,63.599000,0.02,0.98,0.00,1.0,0.0,0.00,0.00,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238063,Emotion-Lenient IS,IS,50.0,2500.0,0.1,0.00002,0.00002,0.20000,0.9,True,0.301384,0.62,0.38,1.00,0.0,0.0,0.00,0.00,1.00
238064,Emotion-Lenient IS,IS,50.0,2500.0,0.1,0.00002,0.00002,0.20000,1.0,True,0.539220,0.52,0.48,1.00,0.0,0.0,0.00,0.82,0.18
238065,Emotion-Lenient IS,IS,50.0,2500.0,0.1,0.00002,0.00002,0.20000,1.0,True,0.282307,0.44,0.56,1.00,0.0,0.0,0.00,1.00,0.00
238066,Emotion-Lenient IS,IS,50.0,2500.0,0.1,0.00002,0.00002,0.20000,1.0,True,0.426726,0.64,0.36,1.00,0.0,0.0,0.00,1.00,0.00


In [4]:
chosen_norm: str = "IS Empirical"
filtered = data[data.EBNorm == chosen_norm]
filtered = filtered[filtered.Norm == "IS"]
filtered = filter(filtered, "z", 50)
filtered = filter(filtered, "eps", 0.00002)
filtered = filter(filtered, "chi", 0.00002)
#filtered = filter(filtered, "alpha", 0.00002)

#new_cols = ["gamma", "eps", "acr", "g_reps", "nice_eps", "AllD", "pDisc", "Disc", "AllC"]
#new_cols = ["gamma", "chi", "acr", "g_reps", "nice_eps", "AllD", "pDisc", "Disc", "AllC"]
new_cols = ["gamma", "alpha", "acr", "g_reps", "nice_eps", "AllD", "pDisc", "Disc", "AllC"]
new_df = pd.DataFrame(columns=new_cols)

gammas = [round(a, 2) for a in np.arange(0, 1.01, 0.1)]
epsilons = [0.00002, 0.0002, 0.002, 0.02, 0.2]
chis = [0.00002, 0.0002, 0.002, 0.02, 0.2]
alphas = [0.00002, 0.0002, 0.002, 0.02, 0.2]

for gamma in gammas:
    #for epsilon in epsilons:
    #for chi in chis:
    for alpha in alphas:
        #super_filtered = filtered[(filtered.gamma == gamma) & (filtered.eps == epsilon)]
        #super_filtered = filtered[(filtered.gamma == gamma) & (filtered.chi == chi)]
        super_filtered = filtered[(filtered.gamma == gamma) & (filtered.alpha == alpha)]
        
        mean_acr: float = super_filtered.acr.mean()
        mean_greps: float = super_filtered.G.mean()
        mean_niceeps: float = super_filtered.nice.mean()
    
        mean_AllC: float = super_filtered.AllC.mean()
        mean_Disc: float = super_filtered.Disc.mean()
        mean_AllD: float = super_filtered.AllD.mean()
        mean_pDisc: float = super_filtered.pDisc.mean()
        #new_df.loc[len(new_df)] = [gamma, epsilon, mean_acr, mean_greps, mean_niceeps, mean_AllD, mean_pDisc, mean_Disc, mean_AllC]
        #new_df.loc[len(new_df)] = [gamma, chi, mean_acr, mean_greps, mean_niceeps, mean_AllD, mean_pDisc, mean_Disc, mean_AllC]
        new_df.loc[len(new_df)] = [gamma, alpha, mean_acr, mean_greps, mean_niceeps, mean_AllD, mean_pDisc, mean_Disc, mean_AllC]

feature: str = "acr" + ":Q"
            
heatmap = alt.Chart(new_df).mark_rect().encode(
    #x='chi:O',
    #x='eps:O',
    x='alpha:O',
    y=alt.Y('gamma:O', sort=alt.EncodingSortField('gamma', order='descending')),
    color=alt.Color(feature, scale=alt.Scale(domain=[0,90]))
).properties(
    height=400,
    width=400,
    title = chosen_norm
)

text = heatmap.mark_text().encode(
    text=alt.Text(feature, format=".2f"),
    color=alt.value('black')
)

#heatmap+text

AttributeError: 'DataFrame' object has no attribute 'nice'

## DIFFERENCE
### gamma in [0, 1] +0.1


In [3]:
data = load_and_process()

chosen_parameter = "eps"

# Initial filtering
filtered = filter(data, "z", 50)
filtered = filter(filtered, "eps", 0.00002) if chosen_parameter != "eps" else filtered
filtered = filter(filtered, "chi", 0.00002) if chosen_parameter != "chi" else filtered
filtered = filter(filtered, "alpha", 0.00002) if chosen_parameter != "alpha" else filtered

# Define columns based on the chosen parameter
new_cols = ["gamma", chosen_parameter, "acr"]
new_df = pd.DataFrame(columns=new_cols)

gammas = [round(a, 2) for a in np.arange(0, 1.01, 0.1)]

# Define values based on the chosen parameter
parameter_values = [0.00002, 0.0002, 0.002, 0.02, 0.2]

for gamma in gammas:
    for parameter_value in parameter_values:
        super_filtered = filtered[(filtered.gamma == gamma) & (filtered[chosen_parameter] == parameter_value)]
        
        mean_acr_nice = super_filtered[super_filtered.EBNorm == "Emotion-Lenient IS"].acr.mean()
        mean_acr_normal = super_filtered[super_filtered.EBNorm == "IS normal"].acr.mean()
        
        new_df.loc[len(new_df)] = [gamma, parameter_value, mean_acr_nice - mean_acr_normal, super_filtered.shape[1]]

# Define feature for Altair chart
feature = "acr:Q"

# Create Altair chart
heatmap = alt.Chart(new_df).mark_rect().encode(
    x=f"{chosen_parameter}:O",
    y=alt.Y('gamma:O', sort=alt.EncodingSortField('gamma', order='descending')),
    color=alt.Color(feature, scale=alt.Scale(domain=[-63, 63], scheme='blueorange'), sort="descending")
).properties(
    height=400,
    width=400,
    title="ACR Difference between EB-IS and IS"
)

# Add text annotations to the chart
text = heatmap.mark_text().encode(
    text=alt.Text(feature, format=".2f"),
    color=alt.value('black')
)

# Display the chart with text annotations
heatmap + text

ValueError: cannot set a row with mismatched columns

## Difference

In [10]:
def heatmap(chosen_parameter, absolute=False):
    data = load_and_process()

    # Initial filtering
    filtered = filter(data, "z", 50)
    filtered = filter(filtered, "eps", 0.00002) if chosen_parameter != "eps" else filtered
    filtered = filter(filtered, "chi", 0.00002) if chosen_parameter != "chi" else filtered
    filtered = filter(filtered, "alpha", 0.00002) if chosen_parameter != "alpha" else filtered
    
    print("filtered size = ", filtered.shape)

    # Define columns based on the chosen parameter
    new_cols = ["gamma", chosen_parameter, "acr", "ACR Difference", "samples", "standard_deviation"]
    new_df = pd.DataFrame(columns=new_cols)

    gammas = [round(a, 2) for a in np.arange(0, 1.01, 0.1)]
    #gammas = filtered.gamma.unique()

    # Define values based on the chosen parameter
    parameter_values = [0.00002, 0.0002, 0.002, 0.02, 0.2]

    for gamma in gammas:
        for parameter_value in parameter_values:
            super_filtered = filtered[(filtered.gamma == gamma) & (filtered[chosen_parameter] == parameter_value)]
            mean_acr_nice = super_filtered[super_filtered.EBNorm == "Emotion-Lenient IS"].acr.mean()
            mean_acr_normal = super_filtered[super_filtered.EBNorm == "IS normal"].acr.mean()
            
            std_acr_nice = super_filtered[super_filtered.EBNorm == "Emotion-Lenient IS"].acr.std()

            new_df.loc[len(new_df)] = [gamma, parameter_value, mean_acr_nice, mean_acr_nice - mean_acr_normal, super_filtered.shape[0], std_acr_nice]

    # Define feature for Altair chart
    feature = "acr:Q" if absolute else "acr_diff:Q"
    domain = [0, 100] if absolute else [-20, 80] 

    # Create Altair chart
    heatmap = alt.Chart(new_df).mark_rect().encode(
        x=f"{chosen_parameter}:O",
        y=alt.Y('gamma:O', sort=alt.EncodingSortField('gamma', order='descending')),
        color=alt.Color(feature, scale=alt.Scale(domain=domain, scheme='blueorange'), sort="descending")
    ).properties(
        height=400,
        width=400,
        title="ACR Difference between ES-IS and IS"
    )

    # Add text annotations to the chart
    text = heatmap.mark_text().encode(
        text=alt.Text(feature, format=".2f"),
        color=alt.value('black')
    )

    # Display the chart with text annotations
    #heatmap + text
    return new_df, (heatmap + text)

In [11]:
chi_df, c = heatmap("chi")
alpha_df, c = heatmap("alpha")
eps_df, c = heatmap("eps")

filtered size =  (76902, 19)
filtered size =  (151330, 19)
filtered size =  (74011, 19)


In [14]:
import plotly.graph_objects as go
import pandas as pd

# Your dataset
df = pd.DataFrame(eps_df)

if df.columns[1] == "eps":
    error = "Action Execution Error" 
elif df.columns[1] == "alpha":
    error = "Reputation Assignment Error"
elif df.columns[1] == "chi":
    error = "Reputation Assessment Error"
else:
    error = ""

z_feature = 'ACR Difference'

# Reshape data for contour plot
z_values = df.pivot_table(index='gamma', columns=df.columns[1], values=z_feature).values
x_values = sorted(df[df.columns[1]].unique())
y_values = sorted(df['gamma'].unique())

# Create contour plot
fig = go.Figure(
    data=go.Contour(
        z=z_values,
        x=x_values,
        y=y_values,
        zmin=-65,
        zmax=65,
        colorscale='RdBu',
        #contours_coloring='lines',
        contours={
            "showlabels":True
        }
    )
)
fig.update_xaxes(type='log')
fig.update_traces(ncontours=25, selector=dict(type='contour'))
title = z_feature + ", Gamma x " + error
fig.update_layout(title=title)
fig.update_layout(
    title=title,
    width=800,  # Set the width of the plot
    height=800,  # Set the height of the plot
    xaxis_title=error,  # Add x-axis label
    yaxis_title='Gamma',  # Add y-axis label
    font=dict(size=16)
)


# Show the plot
fig.show()

In [64]:
data = load_and_process()

value = 0.002
filtered = filter(data, "chi", value)
filtered = filter(filtered, "alpha", value)
filtered = filter(filtered, "eps", value)

new_cols = ["gamma", "Emotion-Based Image Scoring", "Image Scoring", "Samples"]
#new_cols = ["gamma", "Emotion-Based Image Scoring", "std", "Image Scoring"]
#new_cols = ["gamma", "Emotion-Strict Image Scoring", "Emotion-Lenient Image Scoring", "Image Scoring"]
new_df = pd.DataFrame(columns=new_cols)
for gamma in filtered.gamma.unique():
    subset = filtered[filtered.gamma==gamma]
    #print(subset)
    #diff = subset[subset.EBNorm=="IS Empirical"].acr.mean() - subset[subset.EBNorm=="IS normal"].acr.mean()
    entry = [gamma, 
             #subset[subset.EBNorm=="Emotion-Strict IS"].acr.mean(), 
             subset[subset.EBNorm=="Emotion-Lenient IS"].acr.mean(), 
             #subset[subset.EBNorm=="Emotion-Lenient IS"].acr.std(), 
             subset[subset.EBNorm=="IS normal"].acr.mean(),
            subset.shape[0]]
    new_df.loc[len(new_df)] = entry
    
new_df

,gamma,Emotion-Based Image Scoring,Image Scoring,Samples
0,0.00,14.081807,16.600501,300.0
1,0.05,21.348322,14.625768,300.0
2,0.10,24.244356,15.280495,300.0
3,0.15,24.257341,15.746112,300.0
4,0.20,32.963420,13.388435,300.0
5,0.25,36.965993,14.817457,300.0
6,0.30,36.860658,15.207992,300.0
7,0.35,40.907017,13.595427,300.0
8,0.40,43.536340,14.403314,300.0
9,0.45,44.831028,16.120162,300.0


In [61]:
melted = new_df.melt('gamma', var_name='Social Norm', value_name='Average Cooperation Ratio')

#for ix, row in melted:
#    melted.loc[] = new_df.loc[row["gamma"], row["Social Norm"]]
    


In [62]:
alt.Chart(melted).mark_point(filled=True, size=70).encode(
    x='gamma',
    y='Average Cooperation Ratio',
    color='Social Norm:N',
    shape='Social Norm:N'
)

alt.Chart(...)

In [15]:
data = load_and_process()

ebnorm = "Emotion-Lenient IS"

value = 0.002
filtered = filter(data, "chi", value)
filtered = filter(filtered, "alpha", value)
filtered = filter(filtered, "eps", value)
filtered = filter(filtered, "EBNorm", ebnorm)

eps_df = pd.DataFrame(columns=["gamma", "Cooperative EP", "Competitive EP"])
strats_df = pd.DataFrame(columns=["gamma", "AllD", "pDisc", "Disc", "AllC"])
reps_df = pd.DataFrame(columns=["gamma", "good", "bad"])

for gamma in filtered.gamma.unique():
    subset = filtered[filtered.gamma==gamma]
    # CHANGE "MEAN" TO SOMETHING ELSE IN THE DATASET
    
    entry_eps = [gamma, subset.n.mean(), subset.m.mean()]
    entry_strats = [gamma, subset.AllD.mean(), subset.pDisc.mean(), subset.Disc.mean(), subset.AllC.mean()]
    entry_reps = [gamma, subset.G.mean(), subset.B.mean()]
    
    eps_df.loc[len(eps_df)] = entry_eps
    strats_df.loc[len(strats_df)] = entry_strats
    reps_df.loc[len(reps_df)] = entry_reps
    
melted_eps = eps_df.melt('gamma', var_name='emotional profile', value_name='frequency')
melted_strats = strats_df.melt('gamma', var_name='strat', value_name='frequency')
melted_reps = reps_df.melt('gamma', var_name='reputation', value_name='frequency')

In [16]:
melted_strats = melted_strats.drop(melted_strats[melted_strats.strat=="AllC"].index)
melted_strats = melted_strats.drop(melted_strats[melted_strats.strat=="pDisc"].index)
melted_strats

,gamma,strat,frequency
0,0.00,AllD,0.8668
1,0.05,AllD,0.7732
2,0.10,AllD,0.6910
3,0.15,AllD,0.6814
4,0.20,AllD,0.6220
5,0.25,AllD,0.5372
6,0.30,AllD,0.5200
7,0.35,AllD,0.5006
8,0.40,AllD,0.5142
9,0.45,AllD,0.5208


In [41]:
strat_chart = alt.Chart(melted_strats).mark_line(
    interpolate="monotone"
    ).encode(
        x='gamma',
        y='frequency',
        color=alt.Color('strat:N', scale=alt.Scale(scheme="dark2"), sort='descending')
)


reps_chart = alt.Chart(melted_reps).mark_line(
    #size=50,
    #interpolate="monotone",
    strokeDash=[2,2]
).encode(
    x='gamma',
    y='frequency',
    color=alt.Color('reputation', sort='descending')
)

eps_chart = alt.Chart(melted_eps).mark_point(
    size=50,
    filled=True
).encode(
    x='gamma',
    y='frequency',
    shape='emotional profile:N',
    color=alt.Color('emotional profile:N', scale=alt.Scale(scheme="set2"), sort='descending')
)

In [42]:
(strat_chart + eps_chart + reps_chart).resolve_scale(color='independent', shape='independent')

alt.LayerChart(...)

In [28]:
strat_chart + reps_chart

alt.LayerChart(...)

In [20]:
def param_heatmap(chosen_parameter: str, feature: str, norm: str):
    data = load_and_process()

    # Initial filtering
    filtered = filter(data, "z", 50)
    filtered = filter(filtered, "eps", 0.00002) if chosen_parameter != "eps" else filtered
    filtered = filter(filtered, "chi", 0.00002) if chosen_parameter != "chi" else filtered
    filtered = filter(filtered, "alpha", 0.00002) if chosen_parameter != "alpha" else filtered
    
    filtered = filter(filtered, "EBNorm", norm)

    # Define columns based on the chosen parameter
    new_cols = ["gamma", chosen_parameter, "value"]
    new_df = pd.DataFrame(columns=new_cols)

    #gammas = [round(a, 2) for a in np.arange(0.9, 1.01, 0.01)]
    gammas = filtered.gamma.unique()

    # Define values based on the chosen parameter
    parameter_values = [0.00002, 0.0002, 0.002, 0.02, 0.2]

    for gamma in gammas:
        for parameter_value in parameter_values:
            super_filtered = filtered[(filtered.gamma == gamma) & (filtered[chosen_parameter] == parameter_value)]

            feature_mean: float = super_filtered[feature].mean()

            new_df.loc[len(new_df)] = [gamma, parameter_value, feature_mean]

    # Define feature for Altair chart
    view_feature = "value:Q"

    # Create Altair chart
    heatmap = alt.Chart(new_df).mark_rect().encode(
        x=f"{chosen_parameter}:O",
        y=alt.Y('gamma:O', sort=alt.EncodingSortField('gamma', order='descending')),
        color=alt.Color(view_feature, scale=alt.Scale(domain=[0, 1], scheme='blueorange'), sort="ascending")
    ).properties(
        height=400,
        width=400,
        title=chosen_parameter + " x " + feature + " x " + norm
    )

    # Add text annotations to the chart
    text = heatmap.mark_text().encode(
        text=alt.Text(view_feature, format=".2f"),
        color=alt.value('black')
    )

    # Display the chart with text annotations
    #heatmap + text
    return new_df, (heatmap + text)

In [21]:
d, c = param_heatmap("alpha", "AllD", "IS Empirical")
c

alt.LayerChart(...)